In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import geopandas as gpd
from shapely import Point, Polygon, MultiPolygon
from numpy import mean
import matplotlib.pyplot as plt
import ast

In [3]:
def get_data(path, data = 'score'):

    '''

    Extracts person ID and the score of the selected plan for each person in a xml plans output file.

    Input:

    str path - path to plans file

    Output:

    pd.DataFrame - Dataframe with person ID as index and one score column    

    '''

    # Load and parse the XML file
    tree = ET.parse(path)
    root = tree.getroot()
    
    # Initialize an empty list to store the data
    data = []
    
    # Iterate through each person element in the XML
    for person in root.findall('person'):
        person_id = person.attrib['id']
        
        # Find the selected plan
        for plan in person.findall('plan'):
            if plan.attrib.get('selected') == 'yes':
                score = plan.attrib[data]
                data.append({'person_id': person_id, data: float(score)})
    
    # Create a DataFrame with person_id as the index and score as the column
    df = pd.DataFrame(data).set_index('person_id')
    
    # return the DataFrame
    return df


In [4]:
'''

Get scores from selected plans of persons with trips in u5 stop areas


baseCase_scores = get_scores('basecase_filtered_plans_trips_in_u5_stop_areas.xml')

scenario_scores = get_scores('scenario_filtered_plans_trips_in_u5_stop_areas.xml')


baseCase_scores.to_csv('basecase_scores_trips_in_u5_stop_areas.csv')

scenario_scores.to_csv('scenario_scores_trips_in_u5_stop_areas.csv')

'''

# Erstelle DF mit Index person_id und den scores aus BaseCase und Szenario

baseCase_scores = pd.read_csv('basecase_scores_trips_in_u5_stop_areas.csv')

# Eindeutiger Name für Score-Spalte

baseCase_scores = baseCase_scores.rename(columns = {'score': 'base case score'})

# Person ID als Index

baseCase_scores = baseCase_scores.set_index('person_id')



scenario_scores = pd.read_csv('scenario_scores_trips_in_u5_stop_areas.csv')

scenario_scores = scenario_scores.rename(columns = {'score': 'scenario score'})

scenario_scores = scenario_scores.set_index('person_id')


scores_trips_in_stop_area = baseCase_scores.join(scenario_scores)

In [12]:
# Lese trips in U5 Einzugsbereichen ein

trips_in_stop_area = pd.read_csv('trips_in_u5_stop_areas.csv')

In [13]:
#trips_in_stop_area['geometry'] = trips_in_stop_area['geometry'].apply(lambda x: Point(map(float, x.replace('POINT (', '').replace(')', '').split())))

#trips_in_stop_area = gpd.GeoDataFrame(trips_in_stop_area, geometry = 'geometry')

#trips_in_stop_area = trips_in_stop_area.set_crs(epsg='25832')

In [14]:
scoresXtrips = trips_in_stop_area.rename(columns = {'person': 'person_id'})
scoresXtrips = scoresXtrips.set_index('person_id')

scoresXtrips = scoresXtrips.merge(scores_trips_in_stop_area, on='person_id')

scoresXtrips['score difference'] = scoresXtrips['scenario score'] - scoresXtrips['base case score']

In [15]:
stop_areas = scoresXtrips['Haltestelle_area'].unique()

In [16]:
def get_score_diff(stop_areas):

    results = {
        'stop area': [],
        'all trips mean score base case': [],
        'all trips mean score scenario': [],
        'all trips abs difference': [],
        'all trips rel difference': [],
        'u5 trips mean score base case': [],
        'u5 trips mean score scenario': [],
        'u5 trips abs difference': [],
        'u5 trips rel difference': []
    }

    for stop_area in stop_areas:
        all_trips = scoresXtrips[scoresXtrips['Haltestelle_area'] == stop_area]
        
        all_person_id = [pid for pid in all_trips.index.unique()]
        
        all_base_case_scores = [all_trips['base case score'][all_trips.index==pid].iloc[0] for pid in all_trips.index.unique()]
        all_scenario_scores = [all_trips['scenario score'][all_trips.index==pid].iloc[0] for pid in all_trips.index.unique()]

        all_mean_basecase = mean(all_base_case_scores)
        all_mean_scenario = mean(all_scenario_scores)
        all_abs_diff = (all_mean_scenario - all_mean_basecase)
        all_rel_diff = (all_mean_scenario - all_mean_basecase) / all_mean_basecase

        u5_trips = scoresXtrips[(scoresXtrips['Haltestelle_area'] == stop_area) & (scoresXtrips['U5'] == True)]
        
        u5_person_id = [pid for pid in u5_trips.index.unique()]
        u5_base_case_scores = [u5_trips['base case score'][u5_trips.index==pid].iloc[0] for pid in u5_trips.index.unique()]
        u5_scenario_scores = [u5_trips['scenario score'][u5_trips.index==pid].iloc[0] for pid in u5_trips.index.unique()]

        u5_mean_basecase = mean(u5_base_case_scores)
        u5_mean_scenario = mean(u5_scenario_scores)
        u5_abs_diff = (u5_mean_scenario - u5_mean_basecase)
        u5_rel_diff = (u5_mean_scenario - u5_mean_basecase) / u5_mean_basecase

        # Appending results for the current stop_area
        results['stop area'].append(stop_area)
        results['all trips mean score base case'].append(all_mean_basecase)
        results['all trips mean score scenario'].append(all_mean_scenario)
        results['all trips abs difference'].append(all_abs_diff)
        results['all trips rel difference'].append(all_rel_diff * 100)
        results['u5 trips mean score base case'].append(u5_mean_basecase)
        results['u5 trips mean score scenario'].append(u5_mean_scenario)
        results['u5 trips abs difference'].append(u5_abs_diff)
        results['u5 trips rel difference'].append(u5_rel_diff * 100)

    return results

In [17]:
u5_score_stats = pd.DataFrame(get_score_diff(stop_areas))
u5_stops_score_stats = u5_score_stats.set_index('stop area')

In [62]:
# Liste der alten Indexnamen in der gewünschten Reihenfolge
old_index_order = ['Bramfeld', 'Steilshoop', 'Barmbek Nord/Nordheimstraße', 'Sengelmannstraße',
                   'City Nord (Stadtpark)', 'Borgweg', 'Jarrestraße*', 'Beethovenstraße*',
                   'Uhlenhorst*', 'St. Georg*', 'Hauptbahnhof Nord', 'Jungfernstieg',
                   'Stephansplatz/Dammtor', 'Universität*', 'Grindelberg*', 'Hoheluftbrücke',
                   'Gärtnerstraße*', 'UKE*', 'Behrmannplatz*', 'Hagenbecks Tierpark',
                   'Sportplatzring*', 'Stellingen (Arenen)', 'Arenen/Volkspark*']

# Liste der neuen Indexnamen in der gleichen Reihenfolge
new_index_order = ['Bramfeld', 'Steilshoop', 'BarmbekNord', 'B-Sengelmannstraße', 'CityNordStadtpark',
                   'B-Borgweg', 'Jarrestraße', 'Beethovenstraße', 'Uhlenhorst', 'StGeorg',
                   'B-HauptbahnhofNord', 'B-Jungfernstieg', 'B-Stephansplatz', 'Universität',
                   'Grindelberg', 'B-Hoheluftbrücke', 'Gärtnerstraße', 'UniversitätsklinikumE',
                   'Behrmannplatz', 'B-HagenbecksTierpark', 'Sportplatzring', 'Stellingen', 'ArenenVolkspark']

# Erstellen eines Mappings von alten zu neuen Indexnamen
index_mapping = dict(zip(old_index_order, new_index_order))

In [140]:
# Sortiere das DataFrame nach der alten Index-Reihenfolge
u5_stops_score_stats = u5_stops_score_stats.reindex(old_index_order)

# Ersetze die alten Indexnamen durch die neuen
u5_stops_score_stats.index = u5_stops_score_stats.index.map(index_mapping)

# Speicher als CSV
u5_stops_score_stats.to_csv('./ergebnisse/ScoreStats_Pro_Haltestelle.csv')

# Das sortierte und umbenannte DataFrame ist nun fertig
u5_stops_score_stats


,all trips mean score base case,all trips mean score scenario,all trips abs difference,all trips rel difference,u5 trips mean score base case,u5 trips mean score scenario,u5 trips abs difference,u5 trips rel difference
stop area,,,,,,,,
Bramfeld,113.382436,114.027549,0.645113,0.568971,104.502503,107.895080,3.392577,3.246407
Steilshoop,115.400284,116.188682,0.788398,0.683186,107.171647,110.358893,3.187246,2.973964
BarmbekNord,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B-Sengelmannstraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CityNordStadtpark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B-Borgweg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jarrestraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beethovenstraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Uhlenhorst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
def get_transit_stats(stop_areas):

    rel_diff = {
        'stop area': [],
        'all trips travel time': [],
        'u5 trips travel time': [],
        
        'modal share base case': [],
        'modal share scenario': [],
    }

    for stop_area in stop_areas:
            all_trips = transit_data[transit_data['Haltestelle_area'] == stop_area]
            
            all_person_id = [pid for pid in all_trips.index.unique()]
            
            all_base_case_scores = [all_trips['base case score'][all_trips.index==pid].iloc[0] for pid in all_trips.index.unique()]
            all_scenario_scores = [all_trips['scenario score'][all_trips.index==pid].iloc[0] for pid in all_trips.index.unique()]
    
            all_mean_basecase = mean(all_base_case_scores)
            all_mean_scenario = mean(all_scenario_scores)
            all_abs_diff = (all_mean_scenario - all_mean_basecase)
            all_rel_diff = (all_mean_scenario - all_mean_basecase) / all_mean_basecase
    
            u5_trips = transit_data[(transit_data['Haltestelle_area'] == stop_area) & (transit_data['U5'] == True)]
            
            u5_person_id = [pid for pid in u5_trips.index.unique()]
            u5_base_case_scores = [u5_trips['base case score'][u5_trips.index==pid].iloc[0] for pid in u5_trips.index.unique()]
            u5_scenario_scores = [u5_trips['scenario score'][u5_trips.index==pid].iloc[0] for pid in u5_trips.index.unique()]
    
            u5_mean_basecase = mean(u5_base_case_scores)
            u5_mean_scenario = mean(u5_scenario_scores)
            u5_abs_diff = (u5_mean_scenario - u5_mean_basecase)
            u5_rel_diff = (u5_mean_scenario - u5_mean_basecase) / u5_mean_basecase
    
            # Appending results for the current stop_area
            results['stop area'].append(stop_area)
            results['all trips mean score base case'].append(all_mean_basecase)
            results['all trips mean score scenario'].append(all_mean_scenario)
            results['all trips abs difference'].append(all_abs_diff)
            results['all trips rel difference'].append(all_rel_diff * 100)
            results['u5 trips mean score base case'].append(u5_mean_basecase)
            results['u5 trips mean score scenario'].append(u5_mean_scenario)
            results['u5 trips abs difference'].append(u5_abs_diff)
            results['u5 trips rel difference'].append(u5_rel_diff * 100)
    
    return results

In [80]:
def compare_modal_split(df1, df2):
    """
    Vergleicht den Modal Split zwischen zwei Zeitpunkten für jede Haltestelle_area.
    
    Argumente:
    df1 (pandas.DataFrame): Das Dataframe für den ersten Zeitpunkt.
    df2 (pandas.DataFrame): Das Dataframe für den zweiten Zeitpunkt.
    
    Rückgabe:
    Ein pandas.DataFrame, das für jede Haltestelle_area den Modal Split für beide Zeitpunkte und die Änderung enthält.
    """
    # Berechne den Modal Split für jede Haltestelle_area in beiden Dataframes
    modal_split_df1 = pd.DataFrame([calculate_modal_split(df1, area) for area in df1['Haltestelle_area'].unique()])
    modal_split_df2 = pd.DataFrame([calculate_modal_split(df2, area) for area in df2['Haltestelle_area'].unique()])
    
    # Füge die Ergebnisse zusammen und berechne die Änderung
    result_df = pd.merge(modal_split_df1, modal_split_df2, on='Haltestelle_area', suffixes=('_scenario', '_basecase'))
    result_df['Änderung'] = result_df.apply(lambda row: compare_modal_split_dicts(row['modal_split_scenario'], row['modal_split_basecase']), axis=1)
    
    return result_df

def compare_modal_split_dicts(dict1, dict2):
    """
    Vergleicht zwei Modal Split Dictionaries und berechnet die Änderung.
    
    Argumente:
    dict1 (dict): Der Modal Split für den ersten Zeitpunkt.
    dict2 (dict): Der Modal Split für den zweiten Zeitpunkt.
    
    Rückgabe:
    Ein Dictionary, das die Änderung des Modal Splits enthält.
    """
    change = {}
    for mode, value1 in dict1.items():
        value2 = dict2.get(mode, 0)
        change[mode] = value1 - value2
    return change

def calculate_modal_split(df, haltestelle_area):
    """
    Berechnet den Modal Split für eine gegebene Haltestelle_area.
    
    Argumente:
    df (pandas.DataFrame): Das Dataframe mit den Mobilitätsdaten.
    haltestelle_area (str): Der Name der Haltestelle_area, für die der Modal Split berechnet werden soll.
    
    Rückgabe:
    Ein Dictionary mit den Anteilen der einzelnen Verkehrsmittel am Modal Split.
    """
    # Filterung des Dataframes auf die angegebene Haltestelle_area
    haltestelle_data = df[df['Haltestelle_area'] == haltestelle_area]

    haltestelle_data = haltestelle_data.drop_duplicates(subset='trip_id')
    
    # Berechnung der Anteile der einzelnen Verkehrsmittel
    modal_split = haltestelle_data['main_mode'].value_counts(normalize=True)
    
    return {'Haltestelle_area': haltestelle_area, 'modal_split': modal_split.to_dict()}

In [81]:
transit_data_base = pd.read_csv('trips_in_u5_stop_areas_basecase.csv')
transit_data_scenario = pd.read_csv('trips_in_u5_stop_areas.csv')

In [82]:
modal_split_stats = compare_modal_split(transit_data_scenario, transit_data_base)
modal_split_stats = modal_split_stats.set_index('Haltestelle_area')
modal_split_stats.to_csv('./ergebnisse/MSplitStats_Pro_Haltestelle.csv')

In [85]:
modal_split_stats

,modal_split_scenario,modal_split_basecase,Änderung
Haltestelle_area,,,
UKE*,"{'walk': 0.2922543352601156, 'car': 0.28994219...","{'walk': 0.3004162812210916, 'car': 0.29740980...","{'walk': -0.00816194596097597, 'car': -0.00746..."
Jungfernstieg,"{'pt': 0.394487121554451, 'walk': 0.3147311342...","{'pt': 0.3877051648848065, 'walk': 0.315502183...","{'pt': 0.006781956669644484, 'walk': -0.000771..."
Stephansplatz/Dammtor,"{'pt': 0.3505679236710586, 'walk': 0.337210358...","{'pt': 0.34178019981834695, 'walk': 0.34014532...","{'pt': 0.008787723852711649, 'walk': -0.002934..."
Barmbek Nord/Nordheimstraße,"{'pt': 0.2868244436594843, 'walk': 0.257152949...","{'pt': 0.26951607205934297, 'walk': 0.26598375...","{'pt': 0.017308371600141315, 'walk': -0.008830..."
City Nord (Stadtpark),"{'car': 0.3269306154449236, 'pt': 0.2548020384...","{'car': 0.3338557993730408, 'pt': 0.2425548589...","{'car': -0.006925183928117196, 'pt': 0.0122471..."
Sportplatzring*,"{'car': 0.3322867608581894, 'walk': 0.23600209...","{'car': 0.33071690214547356, 'walk': 0.2417582...","{'car': 0.0015698587127158659, 'walk': -0.0057..."
Hagenbecks Tierpark,"{'car': 0.30167173252279633, 'pt': 0.256079027...","{'car': 0.30220197418375094, 'walk': 0.2460136...","{'car': -0.0005302416609546112, 'pt': 0.018417..."
Behrmannplatz*,"{'car': 0.31873373807458805, 'walk': 0.2606244...","{'car': 0.31873373807458805, 'walk': 0.2679965...","{'car': 0.0, 'walk': -0.007372072853425837, 'p..."
Hauptbahnhof Nord,"{'pt': 0.3981644684859904, 'walk': 0.306023217...","{'pt': 0.3900704910063199, 'walk': 0.312530384...","{'pt': 0.0080939774796705, 'walk': -0.00650716..."


-------------------------------------------------------------------------------------------------------------

Travel Times Analyse pro Haltestelle
-

------------------------------------------------------








In [67]:
transit_data_base = pd.read_csv('trips_in_u5_stop_areas_basecase.csv')
transit_data_scenario = pd.read_csv('trips_in_u5_stop_areas.csv')

In [92]:
def compare_trips(basecase_df, scenario_df):
    # Umwandlung von 'trav_time' in Sekunden (timedelta)
    basecase_df['trav_time'] = pd.to_timedelta(basecase_df['trav_time']).dt.total_seconds()
    scenario_df['trav_time'] = pd.to_timedelta(scenario_df['trav_time']).dt.total_seconds()

    # Entfernen von doppelten Trips anhand der 'trip_id' und 'Haltestelle_area'
    basecase_df = basecase_df.drop_duplicates(subset=['trip_id', 'Haltestelle_area'])
    scenario_df = scenario_df.drop_duplicates(subset=['trip_id', 'Haltestelle_area'])
    
    # Gruppieren nach 'Haltestelle_area' und Berechnung der durchschnittlichen Reisezeiten und Distanzen
    basecase_grouped = basecase_df.groupby('Haltestelle_area').agg(
        avg_trav_time_base=('trav_time', 'mean'),
        avg_traveled_distance_base=('traveled_distance', 'mean')
    ).reset_index()

    scenario_grouped = scenario_df.groupby('Haltestelle_area').agg(
        avg_trav_time_scenario=('trav_time', 'mean'),
        avg_traveled_distance_scenario=('traveled_distance', 'mean')
    ).reset_index()

    # Zusammenführen der beiden DataFrames anhand von 'Haltestelle_area'
    merged_df = pd.merge(basecase_grouped, scenario_grouped, on='Haltestelle_area', how='inner')

    # Berechnung der prozentualen Veränderungen
    merged_df['percent_change_trav_time'] = (
        (merged_df['avg_trav_time_scenario'] - merged_df['avg_trav_time_base']) / merged_df['avg_trav_time_base']
    ) * 100

    merged_df['percent_change_traveled_distance'] = (
        (merged_df['avg_traveled_distance_scenario'] - merged_df['avg_traveled_distance_base']) / merged_df['avg_traveled_distance_base']
    ) * 100

    # Formatierung der Prozentzahlen auf 2 Nachkommastellen
    merged_df['percent_change_trav_time'] = merged_df['percent_change_trav_time'].round(2)
    merged_df['percent_change_traveled_distance'] = merged_df['percent_change_traveled_distance'].round(2)

    # Auswahl der relevanten Spalten
    result_df = merged_df[['Haltestelle_area', 'percent_change_trav_time', 'percent_change_traveled_distance']]
    
    # Setze 'Haltestelle_area' als Index
    result_df.set_index('Haltestelle_area', inplace=True)
    
    return result_df


In [93]:
trav_time_stats = compare_trips(transit_data_base, transit_data_scenario)

# Sortiere das DataFrame nach der alten Index-Reihenfolge
trav_time_stats = trav_time_stats.reindex(old_index_order)

# Ersetze die alten Indexnamen durch die neuen
trav_time_stats.index = trav_time_stats.index.map(index_mapping)

# Speicher als CSV
trav_time_stats.to_csv('./ergebnisse/TravTimeStats_Pro_Haltestelle.csv')

trav_time_stats

,percent_change_trav_time,percent_change_traveled_distance
Haltestelle_area,,
Bramfeld,-4.52,1.81
Steilshoop,-5.50,1.01
BarmbekNord,-4.31,0.04
B-Sengelmannstraße,-2.13,-0.60
CityNordStadtpark,-4.06,-1.05
B-Borgweg,-4.90,-0.71
Jarrestraße,-4.51,0.36
Beethovenstraße,-3.83,-0.34
Uhlenhorst,-2.36,-0.35


In [94]:
###### Nur für U5 trips, die im Szenario mit der U5 zurückgelegt werden ######

transit_data_scenario_u5 = transit_data_scenario[transit_data_scenario['U5'] == True]

transit_data_base_u5 = transit_data_base[transit_data_base['trip_id'].isin(transit_data_scenario_u5['trip_id'])]

,Unnamed: 0,person,trip_number,trip_id,dep_time,trav_time,wait_time,traveled_distance,euclidean_distance,main_mode,...,start_y,end_facility_id,end_link,end_x,end_y,first_pt_boarding_stop,last_pt_egress_stop,U5,geometry,Haltestelle_area
28,351,100242,4,100242_4,09:54:40,1641.0,00:00:28,6043,4942,pt,...,5.934297e+06,NaN,1438247360005f,564808.896103,5.939180e+06,BA-JungfernstiegStopFacility,BA-UKEStopFacility,True,POINT (565574.5168225653 5934297.142683576),Jungfernstieg
29,352,100242,4,100242_4,09:54:40,1641.0,00:00:28,6043,4942,pt,...,5.934297e+06,NaN,1438247360005f,564808.896103,5.939180e+06,BA-JungfernstiegStopFacility,BA-UKEStopFacility,True,POINT (565574.5168225653 5934297.142683576),Jungfernstieg
30,353,100242,5,100242_5,18:41:25,5105.0,00:12:35,32338,21499,pt,...,5.939180e+06,NaN,604181030003f,565574.622017,5.960666e+06,BA-UKEStopFacility,000000370103,True,POINT (564808.896103145 5939180.123402637),UKE*
48,564,10038,2,10038_2,08:28:04,1309.0,00:00:51,1975,1130,pt,...,5.936215e+06,NaN,1369654560000f,565358.873217,5.937101e+06,BA-GrindelbergStopFacility,BA-HoheluftbrueckeStopFacility,True,POINT (564656.6543423348 5936215.170218733),Grindelberg*
49,565,10038,2,10038_2,08:28:04,1309.0,00:00:51,1975,1130,pt,...,5.936215e+06,NaN,1369654560000f,565358.873217,5.937101e+06,BA-GrindelbergStopFacility,BA-HoheluftbrueckeStopFacility,True,POINT (564656.6543423348 5936215.170218733),Grindelberg*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501523,1506350,98024,2,98024_2,10:26:48,2135.0,00:01:04,4807,2829,pt,...,5.940206e+06,NaN,3475043700010f,566537.695989,5.937601e+06,BA-SengelmannstrasseStopFacility,BA-JarrestrasseStopFacility,True,POINT (567640.8894927339 5940206.451205621),Sengelmannstraße
501533,1506621,98243,2,98243_2,13:09:57,1228.0,00:00:45,2109,1550,pt,...,5.934228e+06,NaN,2199991740001f,565521.043392,5.934635e+06,BA-Hauptbahnhof_NordStopFacility,BA-JungfernstiegStopFacility,True,POINT (567016.865689811 5934228.154118318),Hauptbahnhof Nord
501536,1506634,98253,7,98253_7,15:25:00,3540.0,00:07:04,21990,18755,pt,...,5.935546e+06,NaN,239931790004f,565752.447320,5.954221e+06,AB-UhlenhorstStopFacility,000000370042,True,POINT (567486.983798315 5935545.613455713),Uhlenhorst*
501572,1507356,98786,5,98786_5,09:39:54,2401.0,00:07:00,11690,10292,pt,...,5.936567e+06,NaN,2206425320003f,572361.963982,5.945651e+06,AB-BeethovenstrasseStopFacility,000000719107,True,POINT (567521.9618086005 5936567.447432051),Beethovenstraße*


In [98]:
trav_time_stats_u5 = compare_trips(transit_data_base_u5, transit_data_scenario_u5)

# Sortiere das DataFrame nach der alten Index-Reihenfolge
trav_time_stats_u5 = trav_time_stats_u5.reindex(old_index_order)

# Ersetze die alten Indexnamen durch die neuen
trav_time_stats_u5.index = trav_time_stats_u5.index.map(index_mapping)

# Speicher als CSV
trav_time_stats_u5.to_csv('./ergebnisse/TravTimeStats_u5_Pro_Haltestelle.csv')

trav_time_stats_u5

/tmp/ipykernel_121736/595380862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basecase_df['trav_time'] = pd.to_timedelta(basecase_df['trav_time']).dt.total_seconds()
/tmp/ipykernel_121736/595380862.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scenario_df['trav_time'] = pd.to_timedelta(scenario_df['trav_time']).dt.total_seconds()


,percent_change_trav_time,percent_change_traveled_distance
Haltestelle_area,,
Bramfeld,-16.62,11.40
Steilshoop,-13.93,4.94
BarmbekNord,-13.59,1.39
B-Sengelmannstraße,-9.61,-3.95
CityNordStadtpark,-14.04,-4.80
B-Borgweg,-13.39,-1.53
Jarrestraße,-16.31,1.82
Beethovenstraße,-14.67,-0.58
Uhlenhorst,-11.36,-1.35
